In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\megaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\megaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 4.2/376.0 MB 20.9 MB/s eta 0:00:18
    --------------------------------------- 8.1/376.0 MB 20.1 MB/s eta 0:00:19
   - -------------------------------------- 12.6/376.0 MB 20.2 MB/s eta 0:00:18
   - -------------------------------------- 17.6/376.0 MB 20.9 MB/s eta 0:00:18
   -- ------------------------------------- 22.5/376.0 MB 21.9 MB/s eta 0:00:17
   -- ------------------------------------- 27.3/376.0 MB 21.9 MB/s eta 0:00:16
   --- ------------------------------------ 33.8/376.0 MB 23.1 MB/s eta 0:00:15
   ---- ----------------------------------- 40.1/376.0 MB 23.8 MB/s eta 0:00:15
   ----- ---------------------------------- 47.7/376.0 MB 25.1 MB/s eta 0:00:14
   ----- ---------------------------------- 55.8/376.0 MB 26.3 MB/s eta 0:00:13
   ------ --------------------------------- 64.0/376.0 MB 27.5 MB/s eta 0:00:12
   ------- -------------------------------- 72.9/37

In [19]:
pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   -- ------------------------------------- 15.7/212.5 MB 99.1 MB/s eta 0:00:02
   ---- ----------------------------------- 24.6/212.5 MB 62.5 MB/s eta 0:00:04
   -------- ------------------------------- 46.9/212.5 MB 78.7 MB/s eta 0:00:03
   ------------- -------------------------- 69.5/212.5 MB 86.9 MB/s eta 0:00:02
   ----------------- ---------------------- 92.8/212.5 MB 92.5 MB/s eta 0:00:02
   --------------------- ----------------- 118.0/212.5 MB 97.8 MB/s eta 0:00:01
   ------------------------- ------------- 140.0/212.5 MB 99.4 MB/s eta 0:00:01
   ----------------------------- -------- 163.1/212.5 MB 100.2 MB/s eta 0:00:01
   --------------------------------- ---- 187.4/212.5 MB 102.4 MB/s eta 0:00:01
   -------------------------------------  211.3/212.5 MB 103.9 MB/s eta 0:00:01
   -------------------------------------  212.3/212.5 MB 102.8 MB/s eta 0:00:01
   -------------------------------------  212.3/2

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [5]:
import torch
import torch.nn as nn

In [7]:
#Load Data
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower()

In [9]:
#Print Tokens
tokens = word_tokenize(text)
print("Total Tokens:", len(tokens))

Total Tokens: 125772


In [11]:
# add a dictionary to map a word and another to map the words back
from collections import Counter

word_counts = Counter(tokens)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)

In [13]:
#Input target sequences
sequence_length = 4  # e.g., "I am going to [predict this]"

data = []
for i in range(len(tokens) - sequence_length):
    input_seq = tokens[i:i + sequence_length - 1]
    target = tokens[i + sequence_length - 1]
    data.append((input_seq, target))

# convert words to indices
def encode(seq): return [word2idx[word] for word in seq]

encoded_data = [(torch.tensor(encode(inp)), torch.tensor(word2idx[target]))
                for inp, target in data]

In [15]:
#Design Model Architecture using Class
class PredictiveKeyboard(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128):
        super(PredictiveKeyboard, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        output = self.fc(output[:, -1, :])  # last LSTM output
        return output

In [17]:
import torch.optim as optim
import random

In [19]:
model = PredictiveKeyboard(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

epochs = 20
for epoch in range(epochs):
    total_loss = 0
    random.shuffle(encoded_data)
    for input_seq, target in encoded_data[:10000]:  # Limit data for speed
        input_seq = input_seq.unsqueeze(0)
        output = model(input_seq)
        loss = criterion(output, target.unsqueeze(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 66130.8354
Epoch 2, Loss: 68044.2236
Epoch 3, Loss: 69431.4519
Epoch 4, Loss: 71257.2095
Epoch 5, Loss: 71353.2787
Epoch 6, Loss: 72548.3559
Epoch 7, Loss: 72651.3637
Epoch 8, Loss: 74788.9088
Epoch 9, Loss: 74966.7768
Epoch 10, Loss: 74432.4795
Epoch 11, Loss: 75721.6624
Epoch 12, Loss: 78366.6934
Epoch 13, Loss: 78033.9548
Epoch 14, Loss: 78392.8573
Epoch 15, Loss: 80387.2584
Epoch 16, Loss: 80361.3355
Epoch 17, Loss: 81650.4595
Epoch 18, Loss: 80571.3293
Epoch 19, Loss: 80528.8822
Epoch 20, Loss: 80735.7273


In [23]:
import torch.nn.functional as F

In [30]:
#Predict words 
def suggest_next_words(model, text_prompt, top_k=3):
    model.eval()
    tokens = word_tokenize(text_prompt.lower())
    if len(tokens) < sequence_length - 1:
        raise ValueError(f"Input should be at least {sequence_length - 1} words long.")

    input_seq = tokens[-(sequence_length - 1):]
    input_tensor = torch.tensor(encode(input_seq)).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1).squeeze()
        top_indices = torch.topk(probs, top_k).indices.tolist()

    return [idx2word[idx] for idx in top_indices]

print("Suggestions:", suggest_next_words(model, "So, are we really at"))

Suggestions: ['the', 'once', 'all']


In [34]:
#Starting next words predictor with tokenizing words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [36]:
#Create input-outpair pairs by splitting texts into sequences and using n-grams
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [40]:
#pad input sequences with equal length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [42]:
#Split sequences
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [44]:
#convert output to one hot encoders
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [54]:
#Build neural network architecture to train model
model = Sequential()
model.add(Embedding(total_words, 100,input_shape=(max_sequence_len - 1,)))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

C:\Users\megaw\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 17, 100)             │         820,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8200)                │       1,238,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,208,800 (8.43 MB)

 Trainable params: 2,208,800 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

None


In [56]:
#Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 143s 46ms/step - accuracy: 0.0622 - loss: 6.5514
Epoch 2/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 146s 47ms/step - accuracy: 0.1199 - loss: 5.5413
Epoch 3/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 170s 37ms/step - accuracy: 0.1485 - loss: 5.1327
Epoch 4/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 142s 37ms/step - accuracy: 0.1654 - loss: 4.7846
Epoch 5/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 141s 36ms/step - accuracy: 0.1869 - loss: 4.4522
Epoch 6/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 111s 37ms/step - accuracy: 0.2057 - loss: 4.1827
Epoch 7/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 141s 37ms/step - accuracy: 0.2332 - loss: 3.8915
Epoch 8/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 110s 37ms/step - accuracy: 0.2653 - loss: 3.6259
Epoch 9/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 111s 37ms/step - accuracy: 0.2975 - loss: 3.3781
Epoch 10/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 142s 37ms/step - accuracy: 0.3346 - loss: 3.1485
Epoch 11/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 144s 37ms/step - accuracy: 0.3717 -

In [58]:
#Next word Prediction output
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
I will leave if they were fastened and
